#### vision : 모델이 이미지를 입력으로 받아 이해하는 능력
- 사진설명, 영수증 분석, 표 읽기, 차트 해석...
- 전용모델 : gpt-image-1(이미지 이해, 생성)

- Response API
- Images API
- Chat Completions API : 예전방식

In [1]:
from dotenv import load_dotenv, find_dotenv
# .env 를 환경변수로 설정
load_dotenv(find_dotenv())

True

In [2]:
# 웹에 있는 특정 이미지 해석

from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role":"user",
        "content":[
            {"type":"input_text", "text":"이 이미지에 무엇이 보이나요?"},
            {"type":"input_image", "image_url":"https://file2.nocutnews.co.kr/newsroom/image/2022/06/15/202206150843431836_0.jpg"},
        ]}
    ]
)
print(response.output_text)

이 이미지에는 일곱 명의 남성이 보이며, 모두 정장 차림입니다. 배경은 흰색이며, 그들은 서로 다소 간격을 두고 서 있습니다. 각자의 스타일이 드러나는 모습이며, 전체적으로 포멀한 느낌을 줍니다.


In [3]:
# 로컬 파일 해석
with open("./image/bag.png","rb") as fr:
    file = client.files.create(file=fr, purpose="vision")

print("업로드된 파일 ID ", file.id)

업로드된 파일 ID  file-4A2AR1foiWHnA5wbMvc5Mi


In [ ]:


from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4o-mini", # vision 지원하는 모델사용해야함
    input=[
        {"role":"user",
        "content":[
            {"type":"input_text", "text":"이 상품 이미지를 보고 쇼핑몰 설명을 작성해줘, 가격대도 추천해"},
            {"type":"input_image", "file_id":file.id},
        ]}
    ]
)
print(response.output_text)

### 제품 설명

**BEICHAO 멀티 스포츠 백** 

모던한 디자인과 실용성을 갖춘 BEICHAO 멀티 스포츠 백을 소개합니다. 이 백은 다채로운 용도로 사용 가능하며, 캠핑, 여행, 운동 등 어떤 활동과도 잘 어울립니다. 

- **소재**: 내구성이 뛰어난 방수 원단 사용으로 다양한 날씨 조건에서도 걱정 없이 사용 가능합니다.
- **디자인**: 세련된 올리브 그린 색상에 브라운 스트랩이 조화롭게 어우러져 멋스러운 인상을 줍니다.
- **수납공간**: 넉넉한 내부 수납공간과 외부 포켓이 있어 소지품을 깔끔하게 정리할 수 있습니다.
- **편리한 이동**: 손잡이와 조절 가능한 어깨끈이 있어 이동 시 편리하게 사용할 수 있습니다.

편안함과 스타일, 두 가지를 모두 갖춘 이 스포츠 백을 통해 당신의 일상에 활력을 더해보세요!

### 가격대
가격은 약 **45,000원**에서 **60,000원** 사이로 추천드립니다. 경쟁력 있는 가격으로 실용성과 스타일을 동시에 잡을 수 있습니다.


In [ ]:
# 이미지 생성(response API)

from openai import OpenAI
import base64

client = OpenAI()

response = client.responses.create(
    model="gpt-4o-mini", # vision 지원하는 모델사용해야함
    input="Generate an image of gray tabby cat hugging an otter with an orange scarf",
    tools=[{"type":"image_generation"}]
)

# 파일 저장
image_data = [ output.result for output in response.output if output.type =="image_generation_call"]

if image_data:
    image_base64 = image_data[0]
    with open("./image/cat_and_otter.png","wb") as f:
        f.write(base64.b64decode(image_base64))

PermissionDeniedError: Error code: 403 - {'error': {'message': 'Your organization must be verified to use the model `gpt-4o-mini`. Please go to: https://platform.openai.com/settings/organization/general and click on Verify Organization. If you just verified, it can take up to 15 minutes for access to propagate.', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [ ]:
# !pip install pillow


[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# 이미지 전용모델

from openai import OpenAI
import base64
from io import BytesIO
from PIL import Image

client = OpenAI()

response = client.images.generate(
    model="gpt-images-1-mini",
    prompt="A cute red panda reading a book, watercolor style",
    size="1024x1024",
    quality="high",
    output_format="png"
)

# 파일 저장
b64 = response.data[0].b64_json
img_bytes = base64.b64decode(b64)
img = Image.open(BytesIO(img_bytes)).convert("RGB")

PermissionDeniedError: Error code: 403 - {'error': {'message': 'Your organization must be verified to use the model `gpt-images-1-mini`. Please go to: https://platform.openai.com/settings/organization/general and click on Verify Organization. If you just verified, it can take up to 15 minutes for access to propagate.', 'type': 'invalid_request_error', 'param': None, 'code': None}}